In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import pandas_datareader as web

In [10]:
start= datetime.datetime(2010,1,1)
end=datetime.datetime(2020,7,16)
NIFTY = web.DataReader('^NSEI','yahoo',start, end)

In [11]:
NIFTY.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,5238.450195,5167.100098,5200.899902,5232.200195,0.0,5232.200195
2010-01-05,5288.350098,5242.399902,5277.149902,5277.899902,0.0,5277.899902
2010-01-06,5310.850098,5260.049805,5278.149902,5281.799805,0.0,5281.799805
2010-01-07,5302.549805,5244.750000,5281.799805,5263.100098,0.0,5263.100098
2010-01-08,5276.750000,5234.700195,5264.250000,5244.750000,0.0,5244.750000


In [12]:
NIFTY['HA_Close']=NIFTY[['Open','High','Close','Low']].mean(axis=1)
NIFTY['ha_open']=np.nan
NIFTY.head()

,High,Low,Open,Close,Volume,Adj Close,HA_Close,ha_open
Date,,,,,,,,
2010-01-04,5238.450195,5167.100098,5200.899902,5232.200195,0.0,5232.200195,5209.662598,NaN
2010-01-05,5288.350098,5242.399902,5277.149902,5277.899902,0.0,5277.899902,5271.449951,NaN
2010-01-06,5310.850098,5260.049805,5278.149902,5281.799805,0.0,5281.799805,5282.712402,NaN
2010-01-07,5302.549805,5244.750000,5281.799805,5263.100098,0.0,5263.100098,5273.049927,NaN
2010-01-08,5276.750000,5234.700195,5264.250000,5244.750000,0.0,5244.750000,5255.112549,NaN


In [14]:
for i in range(0, len(NIFTY)):
        if i == 1:
            NIFTY['ha_open'][i]= ( (NIFTY['Open'][i] + NIFTY['Close'][i] )/ 2)
        else:
            NIFTY['ha_open'][i] = ( (NIFTY['ha_open'][i-1] + NIFTY['HA_Close'][i-1] )/ 2)


In [15]:
NIFTY.info(10)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2579 entries, 2010-01-04 to 2020-07-16
Data columns (total 8 columns):
High         2579 non-null float64
Low          2579 non-null float64
Open         2579 non-null float64
Close        2579 non-null float64
Volume       2579 non-null float64
Adj Close    2579 non-null float64
HA_Close     2579 non-null float64
ha_open      2578 non-null float64
dtypes: float64(8)
memory usage: 261.3 KB


In [16]:
NIFTY['HA_high']=NIFTY.max(axis=1)
NIFTY['HA_low']=NIFTY.min(axis=1)
NIFTY['diff_t']=NIFTY['HA_Close']-NIFTY['ha_open']
NIFTY['has_min']=NIFTY.rolling(10,10)['diff_t'].min()
NIFTY['has_max']=NIFTY.rolling(10,10)['diff_t'].max()
NIFTY['hastoc']=(NIFTY['diff_t']-NIFTY['has_min'])/(NIFTY['has_max']-NIFTY['has_min'])



In [17]:
NIFTY['hastoc'].mean()

0.4961072794432733

In [28]:
#simple hatoc STRATEGY
NIFTY['bst_1']=np.where((NIFTY['diff_t']<0)&(NIFTY['hastoc']<0.1),1,0)

In [29]:
NIFTY.bst_1.value_counts()

0    2100
1     479
Name: bst_1, dtype: int64

In [30]:
NIFTY['st_1']=NIFTY['bst_1'].diff()
NIFTY['log_returns']=np.log(NIFTY['Close']/NIFTY['Close'].shift(1)) 
NIFTY['st_ret_1']=NIFTY['st_1'].shift(1)*NIFTY['log_returns']

In [31]:
st1_aret= round(NIFTY['st_ret_1'].mean() *252, 4)
print(str(st1_aret*100)+'%')

0.66%
